In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#need to run pip install or install cpi package in another way
import cpi



In [ ]:
#Uncomment and run once if cpi index is out of date
#cpi.update()

In [ ]:
nba_salaries= pd.read_csv("salaries_1985to2018.csv")
nba_players=pd.read_csv("players.csv")
frames=[nba_players,nba_salaries]
pd.set_option('display.max_rows',500)



In [ ]:
nba_salaries.head()
nba_salaries.shape

In [ ]:
nba_salaries=nba_salaries[nba_salaries.season_start>=1990]
nba_salaries=nba_salaries[nba_salaries.season_start<=2016]
nba_salaries

In [ ]:
nba_salaries.dtypes

In [ ]:
# salary_col = []
# for index, value in nba_salaries['salary'].iteritems():
#     newsalary = cpi.inflate(value, nba_salaries['season_start'][index] )
#     salary_col.append(newsalary)
# salary_col= pd.Series(salary_col)
# nba_salaries["salary"] = salary_col

In [ ]:
#Account for inflation of these salaries
nba_salaries['salary'] = nba_salaries.apply(lambda x: cpi.inflate(x.salary, x.season_start), axis=1)


In [ ]:
nba_salaries

In [ ]:
nba_salaries = nba_salaries.groupby(['player_id'], as_index=False).mean()
nba_salaries

In [ ]:
nba_salaries=nba_salaries.drop(columns=['season_end', 'season_start'])
nba_salaries


In [ ]:
nba_salaries=nba_salaries.round({"salary":1})

In [ ]:
height_col = []
for index, value in nba_players['height'].iteritems():
    feet, inches = value.split("-")
    print(feet,inches)
    height = int(feet) * 12 + int(inches)
    height_col.append(height)
height_col= pd.Series(height_col)
nba_players["height"] = height_col

In [ ]:
nba_player=nba_players[nba_players.draft_year>='1985']
nba_player.head()

In [ ]:
nba_players=nba_players.drop(columns=[ 'birthPlace', 'college', 'highSchool', 'birthDate',"shoots", 'draft_pick', \
                                     'draft_round', 'draft_year', 'draft_team','weight'])
nba_players=nba_players[['name','_id','position', 'height'
           ,'career_AST', 'career_FG%', 'career_FG3%','career_FT%', 'career_G', 'career_PER', 'career_PTS', 'career_TRB', 'career_WS', 'career_eFG%']]
nba_players.head()


In [ ]:
nba_players=nba_players.dropna(how = 'any')

In [ ]:
nba_players.rename(columns = {"career_FG%" : "careerFG", "career_FG3%" : "careerFG3", "career_FT%" : "careerFT", "career_eFG%":'careereFG', 
                             "career_G":'careerG',"career_PTS":'careerPTS', "career_PER":'careerPER', 
                              "career_TRB":'careerTRB', "career_WS":'careerWS', "career_AST":'careerAST'} , inplace = True)


In [ ]:
nba_players=nba_players.drop_duplicates(subset='name')


In [ ]:
nba_players.shape

In [ ]:
print(nba_players['name'].value_counts())

In [ ]:
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [ ]:
nba_players= tidy_split(nba_players,'position', sep=' and ')
nba_players

In [ ]:
nba_players=nba_players.drop_duplicates(subset='name')

print(nba_players['name'].value_counts())

In [ ]:
nba_players = nba_players[nba_players.careerFG != "-"]
nba_players = nba_players[nba_players.careerFG3 != "-"]
nba_players = nba_players[nba_players.careerFT!= "-"]
nba_players = nba_players[nba_players.careerPER != "-"]
nba_players = nba_players[nba_players.careerTRB != "-"]
nba_players = nba_players[nba_players.careerWS != "-"]
nba_players = nba_players[nba_players.careereFG != "-"]
nba_players.shape

In [ ]:
nba_players

In [ ]:
nba_players.head()

In [ ]:
nba_stats=pd.merge(nba_players, nba_salaries, right_on='player_id',left_on='_id')
nba_stats=nba_stats.drop_duplicates(subset='name')
nba_stats=nba_stats.drop(columns=['_id','player_id'])
nba_stats.sort_values(by=['salary'], ascending=False)


In [ ]:
nba_stats=nba_stats.reset_index(drop=True)
nba_stats.head()

In [ ]:
nba_stats['position']=nba_stats['position'].str.strip()

In [ ]:
dummy_pos=pd.get_dummies(nba_stats['position'], dummy_na=False)
nba_stats=pd.concat([nba_stats,dummy_pos], axis=1)
nba_stats


In [ ]:
th1=nba_stats[(nba_stats.salary >=0 ) & (nba_stats.salary <=4102975.775)]
th2=nba_stats[(nba_stats.salary >=4102975.775) & (nba_stats.salary <=8205951.55)]
th3=nba_stats[(nba_stats.salary >=8205951.55) & (nba_stats.salary <=12308927.325)]
th4=nba_stats[(nba_stats.salary >=12308927.325) & (nba_stats.salary <=16411903.1)]


In [ ]:
nba_stats.salary.max()

In [ ]:
nba_stats['careerFG'] = nba_stats['careerFG'].astype(float)
nba_stats['careerFG3'] = nba_stats['careerFG3'].astype(float)
nba_stats['careerFT'] = nba_stats['careerFT'].astype(float)
nba_stats['careerPER'] = nba_stats['careerPER'].astype(float)
nba_stats['careerTRB'] = nba_stats['careerTRB'].astype(float)
nba_stats['careerWS'] = nba_stats['careerWS'].astype(float)
nba_stats['careereFG'] = nba_stats['careereFG'].astype(float)



In [ ]:
nba_stats.to_csv("NBA_Career_Data.csv", index=False)